Reload modules every time before executing the Python code typed:

In [1]:
%load_ext autoreload
%autoreload 2

Install necessary packages:

In [2]:
%pip install opencv-python numpy scipy filterpy==1.4.5

Note: you may need to restart the kernel to use updated packages.


# Task 1 (exercise 1.2)

Import necessary packages:

In [3]:
import cv2
import numpy as np
from scipy.stats import linregress

import time
import sys
sys.path.append('./')

from sort import Sort

The higher the value, the bigger a moving object should be to be detected

In [4]:
MIN_CONTOUR_AREA = 4000

In [5]:
class Tracker:
    class TrackingObject:
        def __init__(self, initial_contour, tracking_id, ttl) -> None:
            self.tracking_id = tracking_id
            self.ttl = ttl
            self.location_history = []
            self.centroid = Tracker.calc_centroid(initial_contour)
            self.updated = True

            kalman_2d = cv2.KalmanFilter(4, 2)
            kalman_2d.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
            kalman_2d.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
            kalman_2d.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 1e-4
            self.kalman_2d = kalman_2d

        def reset_updated(self):
            self.updated = False

        def update_contour(self, contour):
            self.updated = True
            self.location_history.append(self.centroid)
            self.centroid = Tracker.calc_centroid(contour)
            self.kalman_2d.predict()
            self.kalman_2d.correct(np.array([[np.float32(self.centroid[0])], [np.float32(self.centroid[1])]]))
        
        def update_kalman(self):
            if len(self.location_history) < 5:
                return

            prediction = self.kalman_2d.predict()
            self.centroid = (int(prediction[0]), int(prediction[1]))
            self.location_history.append(self.centroid)

        def x_direction(self):
            if len(self.location_history) < 5:
                return 0

            xs = np.array(self.location_history)[:, 0]
            time = np.arange(len(xs))
    
            # Compute the slope and intercept of the line of best fit
            slope, _ = np.polyfit(time, xs, 1)
    
            # slope > 0 is right, slope < 0 is left, slope = 0 is no movement
            return slope

    def __init__(self, proximity_threshold = 30, object_ttl = 10) -> None:
        self.objects = []
        self.last_tracking_id = 0
        self.proximity_threshold = proximity_threshold
        self.object_ttl = object_ttl
        self.onObjectRemoved = None

    @classmethod
    def calc_centroid(cls, contour):
        """
        Calculates the centroid of a contour.
        """
        M = cv2.moments(contour)
        if M["m00"] == 0:
            # Avoid division by zero
            return None

        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        return (cX, cY)

    def update(self, contours):
        """
        Updates the tracker with a new frame information: contours and centroids.
        """
        original_contours = contours.copy()
        contours = contours.copy()

        # Reset the updated flag for all objects
        [obj.reset_updated() for obj in self.objects]

        # Find the closest object to each contour centroid
        for i, contour in enumerate(contours):
            # Find the centroid of each contour
            centroid = Tracker.calc_centroid(contour)

            if centroid is None:
                continue

            # Find the closest object to the centroid
            closest_object = None
            closest_distance = float('inf')
            for obj in self.objects:
                distance = np.linalg.norm(np.array(obj.centroid) - np.array(centroid))
                if distance < closest_distance:
                    closest_object = obj
                    closest_distance = distance

            # If the closest object is close enough, update it
            if closest_distance < self.proximity_threshold:
                closest_object.update_contour(contour)
                # Remove the contour from the list
                contours[i] = None

        # Add the remaining contours as new objects
        for contour in contours:
            if contour is None:
                continue

            for object in self.objects:
                # If the contour is already have any object, skip it
                if cv2.pointPolygonTest(contour, object.centroid, False) > 0:
                    break
            else:
                # Please note that the first object will have ID 1 will be the whole frame
                self.last_tracking_id = self.last_tracking_id + 1
                print(f"Create a new object with ID: {self.last_tracking_id}")
                self.objects.append(Tracker.TrackingObject(contour, self.last_tracking_id, self.object_ttl))

        [obj.update_kalman() for obj in self.objects if not obj.updated]

        # Remove objects that are not covered by any contour
        for obj in self.objects:
            for contour in original_contours:
                if cv2.pointPolygonTest(contour, obj.centroid, False) > 0:
                    break
            else:
                obj.ttl -= 1
                if obj.ttl <= 0:
                    self.objects.remove(obj)

                    if self.onObjectRemoved is not None:
                        self.onObjectRemoved(obj)


In [17]:
# Open the sample video file:
video = cv2.VideoCapture("Traffic_Laramie_1.mp4")

# Ensure that the file was opened correctly:
assert video.isOpened(), "Can't open the video file"

# Initialize an instance of [Gaussian Mixture-based Background/Foreground Segmentation Algorithm](https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html):
bg_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows=True, history=10000, dist2Threshold=400)

# Instantiate the tracker
tracker = Tracker()

cv2.namedWindow("Cam")

counted_car_ids = set()

def onObjectRemoved(obj):
    if obj.tracking_id not in counted_car_ids and obj.x_direction() < -2 and len(obj.location_history) > 80:
        counted_car_ids.add(obj.tracking_id)

tracker.onObjectRemoved = onObjectRemoved

# Read the video frame by frame:
while True:
    ret, frame = video.read()

    # Check for the last frame
    if not ret:
        break

    # blur the frame to remove noise
    blur = cv2.GaussianBlur(frame, (5,5), 0)

    # Apply the background subtraction algorithm:
    fg_mask = bg_subtractor.apply(blur)

    # Apply morphological operations to remove corruptions:
    _, fg_mask = cv2.threshold(fg_mask, 100, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find the contours of the detected objects:
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out the contours that are too small to be a car:
    contours = [c for c in contours if cv2.contourArea(c) > MIN_CONTOUR_AREA]

    # Convex hull to get clean contours without holes:
    contours = [cv2.convexHull(c) for c in contours]

    # Draw the contours on the original image:
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)

    # # Update the tracker with contours:
    tracker.update(contours)

    # # Draw tracking objects:
    for obj in tracker.objects:
        cv2.putText(frame, f"Car #{obj.tracking_id}", obj.centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, obj.centroid, 2, (0, 0, 255), -1)

    cv2.putText(frame, f'Number of cars go to the city centre: {len(counted_car_ids)}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow("Cam", frame)

    # Slow down the video for debugging purposes 
    # time.sleep(0.05)

    # Press "q" to exit the loop
    if (cv2.waitKey(1) & 0xFF == ord('q')):
        cv2.destroyWindow("Cam")
        cv2.waitKey(1)
        break

Create a new object with ID: 1
Remove object 1
The x_direction is 0
The size of location history is 3
Create a new object with ID: 2
Create a new object with ID: 3
Create a new object with ID: 4


/var/folders/v1/whtp_6_j3p3gwb_dgdz6gkpw0000gn/T/ipykernel_64781/3151435296.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.centroid = (int(prediction[0]), int(prediction[1]))


Remove object 4
The x_direction is 0.030769230769229695
The size of location history is 25
Create a new object with ID: 5
Create a new object with ID: 6
Remove object 2
The x_direction is -6.143946400818774
The size of location history is 185
Remove object 6
The x_direction is -2.6374929418407658
The size of location history is 22
Create a new object with ID: 7
Remove object 7
The x_direction is 2.808683473389355
The size of location history is 35
Create a new object with ID: 8
Remove object 3
The x_direction is -7.69230358843036
The size of location history is 143
Create a new object with ID: 9
Create a new object with ID: 10
Remove object 8
The x_direction is -7.7203007518797
The size of location history is 20
Remove object 9
The x_direction is 0
The size of location history is 4
Create a new object with ID: 11
Remove object 11
The x_direction is 4.466009852216742
The size of location history is 29
Create a new object with ID: 12
Create a new object with ID: 13
Remove object 12
The x